#Imports and installation

## installation

In [1]:
!pip install gradio
!pip install requests
!pip install Pillow
!pip install datasets #To load dataset
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
  

## import

In [2]:
import gradio as gr

In [3]:
import requests
from PIL import Image

In [4]:
from datasets import load_dataset
import pandas as pd

In [5]:
import numpy as np

# Main

In [16]:
import subprocess
subprocess.call(["pip", "install", "-r", "requirements.txt"])

dataset = load_dataset('AbolfazlFekri/SBU_Movie_FP', split = 'train' )
df = dataset.to_pandas()

In [17]:
def fetch_list_posters(movie_names):
    posters = []
    for movie_name in movie_names:
        try:
            # Make a request to the TMDB API to fetch movie information
            response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key=dfdb5f290a8b745bf9c5024c3cf63e7b&query={movie_name}")
            data = response.json()

            # Check if the response contains valid data and results
            if "results" in data and len(data["results"]) > 0:
                # Extract the first movie result from the fetched data
                movie = data["results"][0]

                # Check if the movie result has a poster path
                if "poster_path" in movie and movie["poster_path"]:
                    # Construct the full poster URL
                    poster_path = movie["poster_path"]
                    poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"

                    # Download and append the movie poster to the list
                    image = Image.open(requests.get(poster_url, stream=True).raw)
                    posters.append(poster_url)
                else:
                    # Append None if the movie poster is not available
                    posters.append(None)
            else:
                # Append None if no results are found for the movie
                posters.append(None)

        except Exception as e:
            # Append None in case of any errors
            posters.append(None)

    return posters

In [18]:
#API from TMDB
def fetch_movie_poster(movie_name):
    try:
        # Make a request to the TMDB API to fetch movie information
        response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key=dfdb5f290a8b745bf9c5024c3cf63e7b&query={movie_name}")
        data = response.json()


        # Check if the response contains valid data and results
        if "results" in data and len(data["results"]) > 0:
            # Extract the first movie result from the fetched data
            movie = data["results"][0]

            # Check if the movie result has a poster path
            if "poster_path" in movie and movie["poster_path"]:
                # Construct the full poster URL
                poster_path = movie["poster_path"]
                poster_url = f"https://image.tmdb.org/t/p/w500{poster_path}"

                # Download and return the movie poster
                image = Image.open(requests.get(poster_url, stream=True).raw)
                # print (type(image)) --> <class 'PIL.JpegImagePlugin.JpegImageFile'>
                return poster_url

        # Return a placeholder image if the movie poster is not available
        placeholder_image = Image.new('RGB', (300, 300), color='gray')
        return placeholder_image

    except Exception as e:
        # Return a placeholder image in case of any errors
        placeholder_image = Image.new('RGB', (300, 300), color='gray')
        return placeholder_image

In [19]:
def to_numpy_images_from_urls(image_urls):
    list_of_numpy_images = []
    for url in image_urls:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            image = Image.open(response.raw)
            numpy_image = np.array(image)
            list_of_numpy_images.append(numpy_image)

    return  list_of_numpy_images


In [20]:
def get_label_by_name(movie_name):
  # Set the value you're looking for within the column
  column_name = 'titles'
  retrieve_column = 'clusters'
  column_index = df.columns.get_loc(retrieve_column)

  # Find the row index of the first occurrence of the value within the column
  row_index = df[df[column_name] == movie_name].index[0]

  return df.iloc[row_index, column_index]

In [21]:
# Search function: search by label
def find_samelabel_movie(label):
  column_name = 'clusters'
  retrieve_column = 'titles'
  random_rows_with_given_label = df[df[column_name] == label].sample()
  random_movie_with_given_label = random_rows_with_given_label[retrieve_column]
  # return a list of movie, with given label
  return random_movie_with_given_label.tolist()[0]

In [22]:
def recommend_movie(movie_list):
  recommend_movie_list = []
  i = 0
  while i<5 :
      movie = movie_list[0]
      movie_label = get_label_by_name(movie)
      nominate = find_samelabel_movie(movie_label)
      if (nominate != movie):
        recommend_movie_list.append(nominate)
        i +=1
      else:
        i = 0

  return recommend_movie_list

In [23]:
'''def recommend_movie(movie_list):
  recommend_movie_list = []
  counter = 0
  chosen_label_list = []
  for movie in movie_list:
    movie_label = get_label_by_name(movie)

    if movie_label not in chosen_label_list:
      while (counter<2) :
        nominate = find_samelabel_movie(movie_label)
        if (nominate != movie):
          recommend_movie_list.append(nominate)
          counter +=1
      chosen_label_list.append(movie_label)

    else:
      nominate = find_samelabel_movie(movie_label)
      if (nominate != movie):
        recommend_movie_list.append(nominate)
      chosen_label_list.append(movie_label)


  return recommend_movie_list'''

'def recommend_movie(movie_list):\n  recommend_movie_list = []\n  counter = 0\n  chosen_label_list = []\n  for movie in movie_list:\n    movie_label = get_label_by_name(movie)\n\n    if movie_label not in chosen_label_list:\n      while (counter<2) :\n        nominate = find_samelabel_movie(movie_label)\n        if (nominate != movie):\n          recommend_movie_list.append(nominate)\n          counter +=1\n      chosen_label_list.append(movie_label)\n\n    else:\n      nominate = find_samelabel_movie(movie_label)\n      if (nominate != movie):\n        recommend_movie_list.append(nominate)\n      chosen_label_list.append(movie_label)\n\n\n  return recommend_movie_list'

In [24]:
'''def recommend_movie(movie_list):
  recommend_movie_list = []
  counter = 0
  chosen_label_list = []
  counter = 0
  while len(recommend_movie_list) <6 :
    for movie in movie_list:
      movie_label = get_label_by_name(movie)
      counter += 1
      if counter >5:
        break

  return recommend_movie_list'''

'def recommend_movie(movie_list):\n  recommend_movie_list = []\n  counter = 0\n  chosen_label_list = []\n  counter = 0\n  while len(recommend_movie_list) <6 :\n    for movie in movie_list:\n      movie_label = get_label_by_name(movie)\n      counter += 1\n      if counter >5:\n        break\n\n  return recommend_movie_list'

In [25]:
def gradio_function (selected_movies):

  if len(selected_movies) < 3:
        return "Please select at least Three movies."

  elif len(selected_movies) > 5:
        return "Please select less than 5 movies."

  else:
    # If any movie be selected:
    if selected_movies:
      recommended_movie_posters = []
      movie_list_for_recommendation = recommend_movie(selected_movies)
      for movie in movie_list_for_recommendation:
        recommended_movie_posters.append(fetch_movie_poster(movie))

      recommended_movie_posters = to_numpy_images_from_urls(recommended_movie_posters)
      recommended_movie_posters = np.vstack(tuple(recommended_movie_posters))

      return  recommended_movie_posters

    else:
        return "No movies selected"


In [26]:
# Gradio CheckboxGroup Interface
movie_options = []
for label in range(13):
  movie_options.append(find_samelabel_movie(label))
for label in range(13):
  movie_options.append(find_samelabel_movie(label))

outs = gr.outputs.Image(type='numpy' , label = 'Scroll to see all movies we recommend to you')
iface = gr.Interface( fn=gradio_function, inputs=gr.inputs.CheckboxGroup(choices=movie_options, label="Select between 3 to 5 Movies") , outputs=outs)

# Run the interface
iface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:151: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/inputs.py:154: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Not sure yet

In [ ]:
# Gradio simple Interface
inputs = gr.inputs.Textbox(label="Enter Movie Name")

interface = gr.Interface(fn=get_label_by_name, inputs=inputs, outputs="text")
interface.launch()

In [ ]:
import gradio as gr
import os


def combine(a, b):
    return a + " " + b


def mirror(x):
    return x


with gr.Blocks() as demo:

    txt = gr.Textbox(label="Input", lines=2)
    txt_2 = gr.Textbox(label="Input 2")
    txt_3 = gr.Textbox(value="", label="Output")
    btn = gr.Button(value="Submit")
    btn.click(combine, inputs=[txt, txt_2], outputs=[txt_3])


    gr.Markdown("## Text Examples")
    gr.Examples(
        [["hi", "Adam"], ["hello", "Eve"]],
        [txt, txt_2],
        txt_3,
        combine,
        cache_examples=True,
    )


if __name__ == "__main__":
    demo.launch()


In [ ]:
import gradio as gr
import requests
from PIL import Image

#API from OMDB
def fetch_movie_poster(movie_name):
    try:
        # Make a request to the OMDB API to fetch movie information
        #https://api.themoviedb.org/3/search/movie?api_key=dfdb5f290a8b745bf9c5024c3cf63e7b&query={movie_name} #http://www.omdbapi.com/?apikey=810f8304&t={movie_name}
        response = requests.get(f"https://api.themoviedb.org/3/search/movie?api_key=dfdb5f290a8b745bf9c5024c3cf63e7b&query={movie_name}")
        data = response.json()

        # Check if the response contains valid data
        if "Poster" in data:
            # Extract the poster URL from the fetched data
            poster_url = data['Poster']

            # Download and return the movie poster
            image = Image.open(requests.get(poster_url, stream=True).raw)
            return image
        else:
            # Return a placeholder image if the movie poster is not available
            placeholder_image = Image.new('RGB', (300, 300), color='gray')
            return placeholder_image
    except Exception as e:
        # Return a placeholder image in case of any errors
        placeholder_image = Image.new('RGB', (300, 300), color='gray')

        return placeholder_image
